In [1]:
from functions.netflow2dataframe import *
from functions.portnumber2name import *
from functions.tcpflagletters2names import *
from functions.protocolnumber2name import *

In [2]:
#Will be converted to ipynb eventually

import pandas
import datetime
import numpy



def analyse_df_netflow(df_plus, debug=False, ttl_variation_threshold=4):
    """
    Analysis only top traffic stream

    :param dataframe (df) containing the pcap/pcapng file converted:
    :return (1) print the summary of attack vectors and :
    """
    attack_case = "-1"
    reflection_label = ""
    spoofed_label = ""
    fragment_label = ""

    allpatterns =  [] 
    
    df = df_plus[0]
    summary = df_plus[1]
    
    total_packets = summary[2]
    
    if debug:
        print "Total number packets: " + total_packets
        print "\n###################################\nIDENTIFYING MAIN CHARACTERISTICS:\n###################################"

    top_ip_dst = df.groupby(by=['dst_ip'])['i_packets'].sum().index[-1]
    
    if debug: print "Target (destination) IP: " + top_ip_dst
    

    # Restricting attacks from outside the network!
    # df_filtered = df[(df['dst_ip'] == top_ip_dst) & ~df['src_ip'].str.contains(".".join(top_ip_dst.split('.')[0:2]), na=False) ]

    df_filtered = df[df['dst_ip'] == top_ip_dst]
    
    total_packets_to_target = df_filtered['i_packets'].sum()
    
    if debug: print "Number of packets: " + str(total_packets_to_target)

    ##############################
    ##############################
    while (len(df_filtered) > 0):
        if debug: print "\n###################################################################################################################"
        result = {}
        
        top_ip_proto = df_filtered.groupby(by=['ip_proto'])['i_packets'].sum().index[-1]
        
        result['ip_protocol'] = top_ip_proto
        
        if debug: print "IP protocol used in packets going to target IP: " + str(top_ip_proto)

        df_filtered = df_filtered[df_filtered['ip_proto'] == top_ip_proto]

        # Performing a first filter based on the top_ip_dst (target IP), the source IPs canNOT be from the \16 of the
        # target IP, and the top IP protocol that targeted the top_ip_dst

        ####
        # Calculating the number of packets after the first filter
        total_packets_filtered = df_filtered['i_packets'].sum()
        
        if debug: print "Number of packets: " + str(total_packets_filtered)
        
        result["total_nr_packets"] = total_packets_filtered

        ####
        # For attacks in the IP protocol level
        attack_label = top_ip_proto + "-based attack"
        result["transport_protocol"] = top_ip_proto
        ####
        # For attacks based on TCP or UDP, which have source and destination ports
        if ((top_ip_proto == 'TCP') or (top_ip_proto == 'UDP')):
############ 
###########
# WE STOPPED HERE!!!!
            if debug: print "\n#############################\nPORT FREQUENCY OF REMAINING PACKETS\n##############################"
            ####
            # Calculating the distribution of source ports based on the first filter
            percent_src_ports = df_filtered['src_port'].value_counts().divide(float(total_packets_filtered) / 100)

            if debug: print "SOURCE ports frequency"
            if debug: print percent_src_ports.head()

            ####
            # Calculating the distribution of destination ports after the first filter
            percent_dst_ports = df_filtered['dst_port'].value_counts().divide(float(total_packets_filtered) / 100)
            if debug: print "\nDESTINATION ports frequency"
            if debug: print percent_dst_ports.head()

            #####
            ## WARNING packets are filtered here again#####
            # Using the top 1 (source or destination) port to analyse a pattern of packets
            if (len(percent_src_ports) > 0) and (len(percent_dst_ports) > 0):
                if percent_src_ports.values[0] > percent_dst_ports.values[0]:
                    if debug: print "\nUsing top source port: ", percent_src_ports.keys()[0]
                    df_pattern = df_filtered[df_filtered['src_port'] == percent_src_ports.keys()[0]]
                    result["selected_port"] = "src_" + str(percent_src_ports.keys()[0])
                else:
                    if debug: print "\n Using top dest port: ", percent_dst_ports.keys()[0]
                    df_pattern = df_filtered[df_filtered['dst_port'] == percent_dst_ports.keys()[0]]
                    result["selected_port"] = "dst_" + str(percent_dst_ports.keys()[0])
            else:
                if debug: print 'no top source/dest port'
                return None

            #####
            # Calculating the total number of packets involved in the attack
            pattern_packets = len(df_pattern)
            result["pattern_packet_count"] = pattern_packets

            # WARNING Can be wrong
            #result['raw_attack_size_megabytes'] = (df_pattern['i_bytes'].sum() / 1000000).item()
            #result["pattern_total_megabytes"] = (
            #            df_pattern[df_pattern['fragments'] == 0]['ip_length'].sum() / 1000000).item()
            
            #####
            # Calculating the percentage of the current pattern compared to the raw input file
            representativeness = float(pattern_packets) * 100 / float(total_packets_to_target)
            result["pattern_traffic_share"] = representativeness
            attack_label = 'In %.2f' % representativeness + "\n " + attack_label

            #####
            # Checking the existence of HTTP data
            #http_data = df_pattern['http_data'].value_counts().divide(float(pattern_packets) / 100)
            http_data = ''
            
            #####
            # Checking the existence of TCP flags
            percent_tcp_flags = df_pattern['tcp_flag'].value_counts().divide(float(pattern_packets) / 100)

            #####
            # Calculating the number of source IPs involved in the attack
            ips_involved = df_pattern['src_ip'].unique()
            ######
            if len(ips_involved) < 2:
                if debug: print "\n###################################################################################################################"
                if debug: print "\n###################################################################################################################"
                if debug: print "\n###################################################################################################################"
                if debug: print("\nNO MORE PATTERNS")
                break

            if debug: print(
                        "\n############################\nPATTERN (ATTACK VECTOR) LABEL " + "\n############################")
            attack_label = attack_label + "\n" + str(len(ips_involved)) + " source IPs"
            result["src_ips"] = ips_involved.tolist()

            #print df_pattern['start_time'].min()
            #####
            # Calculating the number of source IPs involved in the attack
            result["start_timestamp"] = df_pattern['start_time'].min()
            result["end_timestamp"] = df_pattern['start_time'].max()


# We do not have that information
#             ####
#             # Calculating the distribution of TTL variation (variation -> number of IPs)
#             ttl_variations = df_pattern.groupby(['src_ip'])['ip_ttl'].agg(np.ptp).value_counts().sort_index()
#             #         if debug: print('TTL variation : NR of source IPs')
#             #         if debug: print(ttl_variations)
#             ips_ttl_greater_4 = ttl_variations.groupby(
#                 np.where(ttl_variations.index > 4, '>4', ttl_variations.index)).sum()
#             #             if debug: print('\n IPs TTL variation >4')
#             #             if debug: print(ips_ttl_greater_4)
#             result["ttl_variation"] = ttl_variations.to_dict()

            ####
            # Calculating the distribution of IP fragments (fragmented -> percentage of packets)
            #percent_fragments = df_pattern['fragments'].value_counts().divide(float(pattern_packets) / 100)
            ####
            # Calculating the distribution of source ports that remains
            percent_src_ports = df_pattern['src_port'].value_counts().divide(float(pattern_packets) / 100)
            result["src_ports"] = percent_src_ports.to_dict()

            ####
            # Calculating the distribution of destination ports after the first filter
            percent_dst_ports = df_pattern['dst_port'].value_counts().divide(float(pattern_packets) / 100)
            result["dst_ports"] = percent_dst_ports.to_dict()

            ####
            # There are 3 possibilities of attacks cases!
            if (percent_src_ports.values[0] == 100):
                df_filtered = df_filtered[df_filtered['src_port'].isin(percent_src_ports.keys()) == False]
                if (len(percent_dst_ports) == 1):
                    # if debug: print("\nCASE 1: 1 source port to 1 destination port") if debug else next
                    port_label = "From " + portnumber2name(
                        percent_src_ports.keys()[0]) + "\n   - Against " + portnumber2name(
                        percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"
                else:
                    # if debug: print("\nCASE 2: 1 source port to a set of destination ports") if debug else next
                    if (percent_dst_ports.values[0] >= 50):
                        port_label = "From " + portnumber2name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                                         0] + "%]" + " and " + portnumber2name(
                            percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_dst_ports.values[
                                         1] + "%]"
                    elif (percent_dst_ports.values[0] >= 33):
                        port_label = "From " + portnumber2name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                                         0] + "%]" + "; " + portnumber2name(percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_dst_ports.values[
                                         1] + "%], and " + portnumber2name(
                            percent_dst_ports.keys()[2]) + "[" + '%.2f' % percent_dst_ports.values[2] + "%]"
                    else:
                        port_label = "From " + portnumber2name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                                         0] + "%]" + "; " + portnumber2name(percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_dst_ports.values[
                                         1] + "%], and " + portnumber2name(
                            percent_dst_ports.keys()[2]) + "[" + '%.2f' % percent_dst_ports.values[2] + "%]"
            else:
                if (len(percent_src_ports) == 1):
                    df_filtered = df_filtered[df_filtered['src_port'].isin(percent_src_ports.keys()) == False]

                    # if debug: print("\nCASE 1: 1 source port to 1 destination port") if debug else next
                    port_label = "Using " + portnumber2name(percent_src_ports.keys()[0]) + "[" + '%.1f' % \
                                 percent_src_ports.values[
                                     0] + "%]" + "\n   - Against " + portnumber2name(
                        percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"


                else:
                    # if debug: print("\nCASE 3: 1 source port to a set of destination ports") if debug else next
                    df_filtered = df_filtered[df_filtered['src_port'].isin(percent_src_ports.keys()) == False]

                    if (percent_src_ports.values[0] >= 50):
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + portnumber2name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                                         0] + "%] and " + portnumber2name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         1] + "%]" + "\n   - Against " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"
                    elif (percent_src_ports.values[0] >= 33):
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + portnumber2name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                                         0] + "%], " + portnumber2name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         1] + "%], and " + portnumber2name(
                            percent_src_ports.keys()[2]) + "[" + '%.2f' % percent_src_ports.values[
                                         2] + "%]" + "\n   - Against " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % \
                                     percent_dst_ports.values[
                                         0] + "%]"
                    else:
                        df_filtered = df_filtered[df_filtered['dst_port'].isin(percent_dst_ports.keys()) == False]
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + portnumber2name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                                         0] + "%], " + portnumber2name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         1] + "%], " + portnumber2name(
                            percent_src_ports.keys()[2]) + "[" + '%.2f' % percent_src_ports.values[
                                         2] + "%]" + "; and " + portnumber2name(
                            percent_src_ports.keys()[3]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         3] + "%]" + "\n   - Against " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"

            ####
            # Testing HTTP request
            if len(http_data) > 0 and ((percent_dst_ports.index[0] == 80) or (percent_dst_ports.index[0] == 443)):
                attack_label = attack_label + "; " + http_data.index[0]

            ####
            # Testing TCP flags
            if (len(percent_tcp_flags) > 0) and (percent_tcp_flags.values[0] > 50):
                attack_label = attack_label + "; TCP flags: " + tcpflagletters2names(
                    percent_tcp_flags.index[0]) + "[" + '%.1f' % percent_tcp_flags.values[0] + "%]"

            ####
#             # IP fragmentation
#             if '1' in percent_fragments.keys():
#                 if (percent_fragments['1'] > 0.3):
#                     fragment_label = "%.2f" % percent_fragments['1'] + "packets with fragments marked"
#                     result["fragmented"] = True

#             ####
#             # IP spoofing (if (more than 0) src IPs had the variation of the ttl higher than a treshold)
#             if '>4' in ips_ttl_greater_4.keys():
#                 if (ips_ttl_greater_4['>4'] > len(ips_involved) * 0.1):
#                     result["spoofed"] = True
#                     spoofed_label = "Likely involving spoofed IPs"
                    ####involved in
                    # Reflection and Amplification
            if percent_src_ports.values[0] >= 1:
                result["reflected"] = True
                reflection_label = "Reflection & Amplification"

            print "\nSUMMARY:\n" \
                  + "- %.2f" % representativeness + "% of the packets targeting " + top_ip_dst + "\n" \
                  + "   - Involved " + str(len(ips_involved)) + " source IP addresses\n" \
                  + "   - Using IP protocol " + protocolnumber2name(top_ip_proto) + "\n" \
                  + "   - " + port_label + "\n" \
                  + "   - " + fragment_label \
                  + "   - " + reflection_label \
                  + "   - " + spoofed_label
            #print result
            #print allpatterns
            #allpatterns["patterns"].append(result)
            allpatterns.append(result)
            
    return allpatterns

In [3]:
df, summary = netflow2dataframe("input_file_for_test/prod-flow-anon-008.nfdump")
df

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,start_time,end_time,time duration,src_ip,dst_ip,src_port,dst_port,ip_proto,tcp_flag,forwarding,...,mpls8,mpls9,mpls10,cl,sl,al,ra,eng,exid,tr
0,2015-10-11 13:44:51,2015-10-11 13:44:51,0,144.108.80.231,150.61.225.0,6925,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000
1,2015-10-11 13:44:51,2015-10-11 13:44:51,0,159.140.178.153,150.61.225.0,30242,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000
2,2015-10-11 13:44:05,2015-10-11 13:44:05,0,212.162.80.249,150.61.225.0,30614,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000
3,2015-10-11 13:44:24,2015-10-11 13:44:24,0,35.161.93.230,150.61.225.0,56841,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000
4,2015-10-11 13:44:20,2015-10-11 13:44:20,0,204.152.74.142,150.61.225.0,64840,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000
5,2015-10-11 13:44:28,2015-10-11 13:44:28,0,195.243.182.225,150.61.225.0,62124,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000
6,2015-10-11 13:44:15,2015-10-11 13:44:15,0,186.148.70.27,150.61.225.0,51149,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000
7,2015-10-11 13:44:09,2015-10-11 13:44:09,0,52.23.167.102,150.61.225.0,12453,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000
8,2015-10-11 13:44:51,2015-10-11 13:44:51,0,195.110.84.153,150.61.225.0,44245,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000
9,2015-10-11 13:44:26,2015-10-11 13:44:26,0,44.39.168.210,150.61.225.0,7600,80,UDP,......,0.0,...,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,0.0.0.0,0/0,1.0,1970-01-01 01:00:00.000


In [4]:
analyse_df_netflow((df,summary), debug=True)

Total number packets: 457123000

###################################
IDENTIFYING MAIN CHARACTERISTICS:
###################################
Target (destination) IP: 150.61.225.0
Number of packets: 457123000.0

###################################################################################################################
IP protocol used in packets going to target IP: UDP
Number of packets: 336054000.0

#############################
PORT FREQUENCY OF REMAINING PACKETS
##############################
SOURCE ports frequency
123      0.029593
19       0.000138
123      0.000028
0        0.000024
19706    0.000004
Name: src_port, dtype: float64

DESTINATION ports frequency
22       0.049722
80       0.017482
31765    0.001162
10929    0.001158
48960    0.001146
Name: dst_port, dtype: float64

 Using top dest port:  22

############################
PATTERN (ATTACK VECTOR) LABEL 
############################


/usr/local/lib/python2.7/dist-packages/pandas/core/ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)



SUMMARY:
- 0.04% of the packets targeting 150.61.225.0
   - Involved 167068 source IP addresses
   - Using IP protocol UDP
   - From a set of (62792) ports, such as port 4596[0.01%], port 36434[0.01%], port 55092[0.01%]; and port 16247[0.01%]
   - Against SSH service port[100.0%]
   -    -    - 

###################################################################################################################
IP protocol used in packets going to target IP: UDP
Number of packets: 113557000.0

#############################
PORT FREQUENCY OF REMAINING PACKETS
##############################
SOURCE ports frequency
123      0.087577
123      0.000083
52262    0.000005
56456    0.000005
24344    0.000004
Name: src_port, dtype: float64

DESTINATION ports frequency
80       0.004305
31765    0.003434
10929    0.003418
48960    0.003382
25734    0.003353
Name: dst_port, dtype: float64

Using top source port:  123

############################
PATTERN (ATTACK VECTOR) LABEL 
####################

[{'dst_ports': {22: 100.0},
  'end_timestamp': '2015-10-11 13:53:51',
  'ip_protocol': 'UDP',
  'pattern_packet_count': 167093,
  'pattern_traffic_share': 0.036553181528822656,
  'selected_port': 'dst_22',
  'src_ips': ['21.243.196.160',
   '202.88.3.3',
   '60.237.38.190',
   '213.8.28.254',
   '21.246.21.161',
   '97.5.11.3',
   '80.199.20.226',
   '189.146.237.193',
   '158.158.14.195',
   '59.217.243.66',
   '59.213.200.125',
   '63.123.255.226',
   '132.200.39.222',
   '176.121.25.224',
   '126.228.54.227',
   '132.69.243.130',
   '156.68.20.255',
   '69.24.20.221',
   '130.14.18.1',
   '126.230.211.63',
   '38.117.209.67',
   '63.176.211.67',
   '46.244.236.221',
   '207.131.193.191',
   '132.86.53.226',
   '97.137.202.62',
   '95.63.250.192',
   '158.137.13.30',
   '37.141.12.224',
   '97.56.89.30',
   '54.100.69.60',
   '176.32.69.126',
   '138.169.182.92',
   '164.149.175.222',
   '95.198.67.98',
   '40.214.182.191',
   '69.246.172.159',
   '189.9.70.33',
   '164.39.185.3',
  